## スレッドを使用してコードを同時に走らせる

In [4]:
use std::thread;
use std::time::Duration;

{
  thread::spawn(|| {
    for i in 1..10 {
      println!("Hi number {} from the spawned thread!", i);
      thread::sleep(Duration::from_millis(1));
    }
  });

  for i in 1..5 {
    println!("Hi number {} from the main thread!", i);
    thread::sleep(Duration::from_millis(1))
  }
}

Hi number 1 from the main thread!
Hi number 1 from the spawned thread!
Hi number 2 from the main thread!
Hi number 2 from the spawned thread!
Hi number 3 from the main thread!
Hi number 3 from the spawned thread!
Hi number 4 from the main thread!


()

In [5]:
()

Hi number 4 from the spawned thread!
Hi number 5 from the spawned thread!
Hi number 6 from the spawned thread!
Hi number 7 from the spawned thread!
Hi number 8 from the spawned thread!
Hi number 9 from the spawned thread!


()

In [6]:
let handle = thread::spawn(|| {
  for i in 1..10 {
    println!("Hi, number {} from the spawned thread!", i);
    thread::sleep(Duration::from_millis(1));
  }
});

for i in 1..5 {
  println!("Hi, number {} from the main thread!", i);
  thread::sleep(Duration::from_millis(1));
}

handle.join().unwrap();

Hi, number 1 from the main thread!
Hi, number 1 from the spawned thread!
Hi, number 2 from the main thread!
Hi, number 2 from the spawned thread!
Hi, number 3 from the main thread!
Hi, number 3 from the spawned thread!
Hi, number 4 from the main thread!
Hi, number 4 from the spawned thread!
Hi, number 5 from the spawned thread!
Hi, number 6 from the spawned thread!
Hi, number 7 from the spawned thread!
Hi, number 8 from the spawned thread!
Hi, number 9 from the spawned thread!


In [7]:
let handle = thread::spawn(|| {
  for i in 1..10 {
    println!("Hi, number {} from the spawned thread!", i);
    thread::sleep(Duration::from_millis(1));
  }
});

handle.join().unwrap();

for i in 1..5 {
  println!("Hi, number {} from the main thread!", i);
  thread::sleep(Duration::from_millis(1));
}

Hi, number 1 from the spawned thread!
Hi, number 2 from the spawned thread!
Hi, number 3 from the spawned thread!
Hi, number 4 from the spawned thread!
Hi, number 5 from the spawned thread!
Hi, number 6 from the spawned thread!
Hi, number 7 from the spawned thread!
Hi, number 8 from the spawned thread!
Hi, number 9 from the spawned thread!
Hi, number 1 from the main thread!
Hi, number 2 from the main thread!
Hi, number 3 from the main thread!
Hi, number 4 from the main thread!


()

In [9]:
let v = vec![1,2,3];

let handle = thread::spawn(|| {
  println!("Here's a vector: {:?}", v);
});

handle.join().unwrap();

Error: closure may outlive the current function, but it borrows `v`, which is owned by the current function

In [10]:
let v = vec![1,2,3];

let handle = thread::spawn(move || {
  println!("Here's a vector: {:?}", v);
});

handle.join().unwrap();

Here's a vector: [1, 2, 3]


In [11]:
let v = vec![1,2,3];

let handle = thread::spawn(move || {
  println!("Here's a vector: {:?}", v);
});

drop(v);

handle.join().unwrap();

Error: use of moved value: `v`

## メッセージ受け渡しを使ってスレッド間でデータを転送する

In [12]:
use std::sync::mpsc;

let (tx, rx) = mpsc::channel();

Error: consider importing one of these items

In [13]:
use std::thread;
use std::sync::mpsc;

let (tx, rx) = mpsc::channel();
thread::spawn(move || {
  let val = String::from("hi");
  tx.send(val).unwrap();
})

JoinHandle { .. }

In [15]:
let (tx, rx) = mpsc::channel();
thread::spawn(move || {
  let val = String::from("hi");
  tx.send(val).unwrap();
});

let received = rx.recv().unwrap();
println!("Got: {}", received);

Got: hi


In [16]:
let (tx, rx) = mpsc::channel();
thread::spawn(move || {
  let val = String::from("hi");
  tx.send(val).unwrap();
  println!("val is {}", val);
});

let received = rx.recv().unwrap();
println!("Got: {}", received);

Error: borrow of moved value: `val`

In [18]:
let (tx, rx) = mpsc::channel();

thread::spawn(move || {
  let vals = vec![
    String::from("hi"),
    String::from("from"),
    String::from("the"),
    String::from("thread"),
  ];
  for val in vals {
    tx.send(val).unwrap();
    thread::sleep(Duration::from_secs(1));
  }
});


for received in rx {
  println!("Got: {}", received);
}


Got: hi
Got: from
Got: the
Got: thread


()

In [20]:
let (tx, rx) = mpsc::channel();
let tx1 = mpsc::Sender::clone(&tx);

thread::spawn(move || {
  let vals = vec![
    String::from("hi"),
    String::from("from"),
    String::from("the"),
    String::from("thread"),
  ];
  for val in vals {
    tx1.send(val).unwrap();
    thread::sleep(Duration::from_secs(1));
  }
});

thread::spawn(move || {
  let vals = vec![
    String::from("more"),
    String::from("message"),
    String::from("for"),
    String::from("you"),
  ];

  for val in vals {
    tx.send(val).unwrap();
    thread::sleep(Duration::from_secs(1));
  }
});

for received in rx {
  println!("Got: {}", received);
}


Got: hi
Got: more
Got: message
Got: from
Got: the
Got: for
Got: you
Got: thread


()

In [21]:
let (tx, rx) = mpsc::channel();
let tx1 = mpsc::Sender::clone(&tx);

thread::spawn(move || {
  let vals = vec![
    String::from("hi"),
    String::from("from"),
    String::from("the"),
    String::from("thread"),
  ];
  for val in vals {
    tx.send(val).unwrap();
    thread::sleep(Duration::from_secs(1));
  }
});

thread::spawn(move || {
  let vals = vec![
    String::from("more"),
    String::from("message"),
    String::from("for"),
    String::from("you"),
  ];

  for val in vals {
    tx.send(val).unwrap();
    thread::sleep(Duration::from_secs(1));
  }
});

for received in rx {
  println!("Got: {}", received);
}


Error: use of moved value: `tx`